In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os


def read_pts(fname):
    f = open(fname)
    lines = f.readlines()
    n_points = int(lines[1].split(':')[1].strip())
    
    data_lines = lines[3:(n_points+3):1]
    points = []
    for line in data_lines:
        coords = line.strip().split()
        points.append(list(map(float, coords)))
        
    f.close()    
    return np.array(points)

In [ ]:
path = ["./FGNET/images/052A20.JPG", "./FGNET/points/052a20.pts"]

face_list, age_list, pid_list = [], [], []

img = cv2.imread(path[0], cv2.IMREAD_COLOR)

cv2.imshow("RGB Image", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread(path[0], cv2.IMREAD_GRAYSCALE)

cv2.imshow("Grayscale Image", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
landmarks = read_pts(path[1])

point_color = (0, 0, 255)
point_radius = 5
point_thickness = -1

cnt = 0
for point in landmarks:
    cnt += 1
    cv2.circle(img, point.astype(int), point_radius, point_color, point_thickness)

cv2.imshow("Facial Landmarks", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread(path[0], cv2.IMREAD_GRAYSCALE)

landmarks = read_pts(path[1])
pupils = landmarks[[31, 36], :]

cnt = 0
for point in pupils:
    cnt += 1
    cv2.circle(img, point.astype(int), point_radius, point_color, point_thickness)

cv2.imshow("Before Head Rotation Correction", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread(path[0], cv2.IMREAD_GRAYSCALE)

landmarks = read_pts(path[1])
pupils = landmarks[[31, 36], :]

delta_x, delta_y = pupils[1] - pupils[0]
angle = np.arctan(delta_y / delta_x) * 180 / np.pi

h, w = img.shape
(cx, cy) = (h // 2, w // 2)

rotation_matrix = cv2.getRotationMatrix2D(center=(cx, cy), angle=angle, scale=1)
img = cv2.warpAffine(img, rotation_matrix, dsize=(img.shape[1], img.shape[0]))

landmarks = np.transpose(np.dot(rotation_matrix[:, 0:2], np.transpose(landmarks))) + np.transpose(rotation_matrix[:, 2])
pupils = np.transpose(np.dot(rotation_matrix[:, 0:2], np.transpose(pupils))) + np.transpose(rotation_matrix[:, 2])

x_min, y_min = np.min(landmarks[:, 0]).astype(int), np.min(landmarks[:, 1]).astype(int)
x_max, y_max = np.max(landmarks[:, 0]).astype(int), np.max(landmarks[:, 1]).astype(int)

cv2.imshow("After Head Rotation Correction", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread(path[0], cv2.IMREAD_GRAYSCALE)

landmarks = read_pts(path[1])
pupils = landmarks[[31, 36], :]

delta_x, delta_y = pupils[1] - pupils[0]
angle = np.arctan(delta_y / delta_x) * 180 / np.pi

h, w = img.shape
(cx, cy) = (h // 2, w // 2)

rotation_matrix = cv2.getRotationMatrix2D(center=(cx, cy), angle=angle, scale=1)
img = cv2.warpAffine(img, rotation_matrix, dsize=(img.shape[1], img.shape[0]))

landmarks = np.transpose(np.dot(rotation_matrix[:, 0:2], np.transpose(landmarks))) + np.transpose(rotation_matrix[:, 2])
pupils = np.transpose(np.dot(rotation_matrix[:, 0:2], np.transpose(pupils))) + np.transpose(rotation_matrix[:, 2])

x_min, y_min = np.min(landmarks[:, 0]).astype(int), np.min(landmarks[:, 1]).astype(int)
x_max, y_max = np.max(landmarks[:, 0]).astype(int), np.max(landmarks[:, 1]).astype(int)

height = y_max - y_min + 1
width = x_max - x_min + 1
if height > width:
    print("case1")
    diff = height - width
    pad = diff // 2
    img = img[y_min:(y_max + 1), (x_min - pad):(x_max + 1 + (diff - pad))]
elif height < width:
    print("case2")
    diff = width - height
    pad = diff // 2
    img = img[(y_min - pad):(y_max + 1 + (diff - pad)), x_min:(x_max + 1)]
else:
    print("case3")
    img = img[y_min:(y_max + 1), x_min:(x_max + 1)]

cv2.imshow("Cropped Image", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
img = clahe.apply(img)

cv2.imshow("CLAHE", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
img = cv2.resize(src=img, dsize=(48, 48), interpolation=cv2.INTER_LINEAR)

cv2.imshow("Resized Image", img)
cv2.waitKey()
cv2.destroyAllWindows()